In [1]:
import os 
import numpy as np
import cv2
import pandas as pd
from PIL import ImageFont, ImageDraw, Image
import jsonlines
import PyPDF2
import pdf2image
from PyPDF2 import PdfFileReader
from matplotlib.patches import  Rectangle

In [2]:
data_fintabnet_path =  "D:/fintabnet/fintabnet/"
subdir = "train"
structure_filename  = "FinTabNet_1.0.0_cell_" + subdir + ".jsonl"
detection_filename = "FinTabNet_1.0.0_table_" + subdir +  ".jsonl"
structure_filepath = os.path.join(data_fintabnet_path, structure_filename)
detection_filepath = os.path.join(data_fintabnet_path, detection_filename)
print(f'detection_filepath :{structure_filepath}')
with open(structure_filepath, "r") as f:
    structure_lines = f.readlines()

detection_filepath :D:/fintabnet/fintabnet/FinTabNet_1.0.0_cell_train.jsonl


In [14]:
i = 0
data_frame = {}
with jsonlines.open(detection_filepath,'r') as f:
# with jsonlines.open('FinTabNet_1.0.0_table_example.jsonl','r') as f:
    for data in f:
        if i > 1 :
            break
        file_name = data['filename']
        print(f'file_name :{file_name}')
        if file_name in data_frame:
                 = data['bbox']
            data_frame[file_name].append(box) 
        else:
            box = data['bbox']  
            update = {file_name : [box]}
            data_frame.update(**update)
        i = i +1 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)